<a href="https://colab.research.google.com/github/kphopthorn/Multi-Label/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Oct 10 11:23:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers iterative-stratification tensorflow_addons sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [ ]:
train_path = "/content/drive/MyDrive/IS_multi-label/train_data.csv"
test_path = "/content/drive/MyDrive/IS_multi-label/test_data.csv"

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df.columns

Index(['Unnamed: 0', 'type', 'comment', 'comm_norm', 'PM2.5', 'การเดินทาง',
       'กีดขวาง', 'คนจรจัด', 'คลอง', 'ความปลอดภัย', 'ความสะอาด', 'จราจร',
       'ต้นไม้', 'ถนน', 'ทางเท้า', 'ท่อระบายน้ำ', 'น้ำท่วม', 'ป้าย',
       'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม', 'สะพาน', 'สัตว์จรจัด', 'สายไฟ',
       'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน', 'แสงสว่าง'],
      dtype='object')

In [ ]:
target_list = ['PM2.5', 'การเดินทาง','กีดขวาง', 'คนจรจัด', 'คลอง', 'ความปลอดภัย', 'ความสะอาด', 'จราจร',
       'ต้นไม้', 'ถนน', 'ทางเท้า', 'ท่อระบายน้ำ', 'น้ำท่วม', 'ป้าย',
       'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม', 'สะพาน', 'สัตว์จรจัด', 'สายไฟ',
       'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน', 'แสงสว่าง']

In [ ]:
train_df = train_df[['comm_norm', 'PM2.5', 'การเดินทาง',
       'กีดขวาง', 'คนจรจัด', 'คลอง', 'ความปลอดภัย', 'ความสะอาด', 'จราจร',
       'ต้นไม้', 'ถนน', 'ทางเท้า', 'ท่อระบายน้ำ', 'น้ำท่วม', 'ป้าย',
       'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม', 'สะพาน', 'สัตว์จรจัด', 'สายไฟ',
       'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน', 'แสงสว่าง']]

test_df = test_df[['comm_norm', 'PM2.5', 'การเดินทาง',
       'กีดขวาง', 'คนจรจัด', 'คลอง', 'ความปลอดภัย', 'ความสะอาด', 'จราจร',
       'ต้นไม้', 'ถนน', 'ทางเท้า', 'ท่อระบายน้ำ', 'น้ำท่วม', 'ป้าย',
       'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม', 'สะพาน', 'สัตว์จรจัด', 'สายไฟ',
       'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน', 'แสงสว่าง']]

In [ ]:
train_df.head()

,comm_norm,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,ความสะอาด,จราจร,ต้นไม้,...,ป้ายจราจร,ร้องเรียน,สอบถาม,สะพาน,สัตว์จรจัด,สายไฟ,ห้องน้ำ,เสนอแนะ,เสียงรบกวน,แสงสว่าง
0,สะพาน ไทย ญี่ปุ่น มุ่งหน้า บ่อน ไก่ สะพาน เหลื...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,‘ ข้อมูล บรรยาย ประกอบการ ปักหมุด ปัญหา แจ้ง แ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,ปัญหา งานก่อสร้าง อาคาร ที่จอดรถ คนงาน เชื่อมเ...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,ต้นไม้ ยื่น ออกจาก บ้าน ถนน เจ้าของ ต้นไม้ กีด...,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,สะพานลอย ป้ายรถเมล์ ห้าง เพชรเกษม ปาก ทางเข้า ...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [ ]:
MAX_LEN = 100
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 2
LEARNING_RATE = 1e-05

In [ ]:
from transformers import (CamembertTokenizer,AutoModel)

# Load pre-trained tokenizer
tokenizer = CamembertTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased',
                                  revision='main')


In [ ]:
#["monsoon-nlp/bert-base-thai", "bert-base-multilingual-uncased", "Geotrend/bert-base-th-cased","airesearch/wangchanberta-base-att-spm-uncased"]

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained('Geotrend/bert-base-th-cased')
bert = AutoModel.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['comm_norm']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            #'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.utils import indexable, _safe_indexing
from sklearn.utils.validation import _num_samples
from sklearn.model_selection._split import _validate_shuffle_split
from itertools import chain

def multilabel_train_test_split(*arrays,
                                test_size=None,
                                train_size=None,
                                random_state=None,
                                shuffle=True,
                                stratify=None):
    """
    Train test split for multilabel classification. Uses the algorithm from:
    'Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of Multi-Label Data'.
    """
    if stratify is None:
        return train_test_split(*arrays, test_size=test_size,train_size=train_size,
                                random_state=random_state, stratify=None, shuffle=shuffle)

    assert shuffle, "Stratified train/test split is not implemented for shuffle=False"

    n_arrays = len(arrays)
    arrays = indexable(*arrays)
    n_samples = _num_samples(arrays[0])
    n_train, n_test = _validate_shuffle_split(
        n_samples, test_size, train_size, default_test_size=0.25
    )
    cv = MultilabelStratifiedShuffleSplit(test_size=n_test, train_size=n_train, random_state=123)
    train, test = next(cv.split(X=arrays[0], y=stratify))

    return list(
        chain.from_iterable(
            (_safe_indexing(a, train), _safe_indexing(a, test)) for a in arrays
        )
    )

In [ ]:
train_df.reset_index(drop=True,inplace=True)
train_df

,comm_norm,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,ความสะอาด,จราจร,ต้นไม้,...,ป้ายจราจร,ร้องเรียน,สอบถาม,สะพาน,สัตว์จรจัด,สายไฟ,ห้องน้ำ,เสนอแนะ,เสียงรบกวน,แสงสว่าง
0,สะพาน ไทย ญี่ปุ่น มุ่งหน้า บ่อน ไก่ สะพาน เหลื...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,‘ ข้อมูล บรรยาย ประกอบการ ปักหมุด ปัญหา แจ้ง แ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,ปัญหา งานก่อสร้าง อาคาร ที่จอดรถ คนงาน เชื่อมเ...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,ต้นไม้ ยื่น ออกจาก บ้าน ถนน เจ้าของ ต้นไม้ กีด...,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,สะพานลอย ป้ายรถเมล์ ห้าง เพชรเกษม ปาก ทางเข้า ...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83962,ปล่อย สัญญาณไฟจราจร เหมาะสม รถ สัญจร ไฟเลี้ยว ...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
83963,เบส ซอย กรุงเทพ นนทบุรี ถนน กรุงเทพ นนทบุรี ซื...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83964,ปัญหา หน้าบ้าน รถ จอด ทางเข้าออก หน้าบ้าน เข้า...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83965,บริเวณ สะพานข้าม แยก โบว์ รถเก่า จอด สะพาน เป็...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
X = train_df[['comm_norm']]
Y = train_df[['PM2.5', 'การเดินทาง', 'กีดขวาง', 'คนจรจัด', 'คลอง',
       'ความปลอดภัย', 'ความสะอาด', 'จราจร', 'ต้นไม้', 'ถนน', 'ทางเท้า',
       'ท่อระบายน้ำ', 'น้ำท่วม', 'ป้าย', 'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม',
       'สะพาน', 'สัตว์จรจัด', 'สายไฟ', 'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน',
       'แสงสว่าง']]

In [ ]:
train_X, val_X, train_y, val_y = multilabel_train_test_split(X,Y,stratify=Y, test_size=0.15)

In [ ]:
train_data = pd.concat([train_X, train_y], axis=1).reset_index(drop=True)
val_data = pd.concat([val_X, val_y], axis=1).reset_index(drop=True)

In [ ]:
train_dataset = CustomDataset(train_data, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_data, tokenizer, MAX_LEN)

In [ ]:
test_word = 'ต้นไม้ ขวาง คลอง'
inputs_test = tokenizer.encode_plus(
            test_word,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
print(inputs_test)

{'input_ids': tensor([[   5,   10, 2414,   10, 4933,   10, 1362,    6,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
train_data['comm_norm'][0]

'สะพาน ไทย ญี่ปุ่น มุ่งหน้า บ่อน ไก่ สะพาน เหลือง ทางลง สวนลุม หลุม ลึก หลุม'

In [ ]:
train_dataset[0]

{'input_ids': tensor([   5,   10, 1981,   10,  109,   10,  569,   10, 5770,   10, 9904,   10,
         1305,   10, 1981,   10, 2776,   10,   75,  154,   10, 1079, 6016,   76,
           10, 3980,   10, 1844,   10, 3980,    6,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
print(len(train_dataset))
print(len(valid_dataset))

71360
12607


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
# unfreeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self,bert):
        super(BERTClass, self).__init__()
        self.bert_model = bert
        self.dropout = torch.nn.Dropout(0.2)
        self.linear = torch.nn.Linear(768, 16)
        self.relu = nn.ReLU()
        self.linear2 = torch.nn.Linear(16, 24)

    def forward(self, input_ids, attn_mask):
        _, cls_hs = self.bert_model(
            input_ids,
            attention_mask=attn_mask,return_dict=False
        )
        #output= self.dropout(output.pooler_output)
        output = self.dropout(cls_hs)
        output = self.linear(output)
        output = self.relu(output)
        output = self.linear2(output)
        return output

model = BERTClass(bert)
model.to(device)

BERTClass(
  (bert_model): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        #token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            #token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss
      valid_loss = valid_loss
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/IS_multi-label/curr_ckpt/checkpoint_wangchan.pt" #[checkpoint.pt]
best_model_path = "/content/drive/MyDrive/IS_multi-label/best_model_wangchan.pt" #[best_model.pt]

In [ ]:
model, optimizer, start_epoch, valid_loss_min = load_ckp(best_model_path, BERTClass(bert), optimizer)
model.to(device)

BERTClass(
  (bert_model): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [ ]:
trained_model = train_model(5, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

In [ ]:

example = 'ไฟแดง เลี้ยว ขวา เข้า ซอย เสีย ทำ ให้ รถ เลี้ยว ฝ่า ไฟ แดง ใน ตอน ทางตรง ไฟ เขียว อันตราย มาก ครับ'
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=False,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
        )
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    #token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    for column_name, prediction in zip(train_data.columns[1:], final_output[0]):
      print(f'{column_name}  -->  {prediction}')


PM2.5  -->  0.0010968151036649942
การเดินทาง  -->  0.020636050030589104
กีดขวาง  -->  0.000543104310054332
คนจรจัด  -->  0.013098056428134441
คลอง  -->  0.00016120389045681804
ความปลอดภัย  -->  0.5707849860191345
ความสะอาด  -->  0.30400821566581726
จราจร  -->  0.4640367329120636
ต้นไม้  -->  0.0409204326570034
ถนน  -->  0.14564712345600128
ทางเท้า  -->  0.13617751002311707
ท่อระบายน้ำ  -->  0.0010126911802217364
น้ำท่วม  -->  0.04871528223156929
ป้าย  -->  0.002142710844054818
ป้ายจราจร  -->  0.05767016485333443
ร้องเรียน  -->  0.07281289994716644
สอบถาม  -->  0.02479746751487255
สะพาน  -->  0.011294222436845303
สัตว์จรจัด  -->  0.0007306269835680723
สายไฟ  -->  0.1346396654844284
ห้องน้ำ  -->  0.0021390211768448353
เสนอแนะ  -->  0.03578519448637962
เสียงรบกวน  -->  0.07635799050331116
แสงสว่าง  -->  0.04503266140818596


In [ ]:
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)
test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
def test_pred():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_data_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            #token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets


In [ ]:
cols = ['PM2.5', 'การเดินทาง', 'กีดขวาง', 'คนจรจัด', 'คลอง',
       'ความปลอดภัย', 'ความสะอาด', 'จราจร', 'ต้นไม้', 'ถนน', 'ทางเท้า',
       'ท่อระบายน้ำ', 'น้ำท่วม', 'ป้าย', 'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม',
       'สะพาน', 'สัตว์จรจัด', 'สายไฟ', 'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน',
       'แสงสว่าง']

In [ ]:
outputs, targets = test_pred()

In [ ]:
print(len(outputs))
print(len(targets))

21087
21087


In [ ]:
output_df = pd.DataFrame(outputs, columns=cols)

In [ ]:
output_df

,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,ความสะอาด,จราจร,ต้นไม้,ถนน,...,ป้ายจราจร,ร้องเรียน,สอบถาม,สะพาน,สัตว์จรจัด,สายไฟ,ห้องน้ำ,เสนอแนะ,เสียงรบกวน,แสงสว่าง
0,0.003937,0.004812,0.038515,6.553722e-02,0.009537,0.772956,0.011639,0.003235,0.192054,0.935340,...,0.086371,0.021169,0.005350,0.005296,0.000165,0.017432,0.008384,0.001770,0.007163,0.790552
1,0.000023,0.004736,0.090009,8.600747e-07,0.037590,0.050612,0.020160,0.000235,0.011389,0.021671,...,0.001342,0.065973,0.002024,0.005614,0.067535,0.038357,0.023863,0.000287,0.000434,0.013299
2,0.049860,0.028484,0.005496,5.528663e-05,0.041242,0.082963,0.674302,0.046487,0.832040,0.050445,...,0.002258,0.020879,0.002382,0.013547,0.032457,0.091024,0.007557,0.001461,0.002870,0.004427
3,0.024742,0.038229,0.018608,5.264437e-04,0.013332,0.052319,0.044839,0.018269,0.802698,0.874716,...,0.000877,0.039592,0.003464,0.014551,0.001419,0.015859,0.036866,0.000102,0.002694,0.064694
4,0.000472,0.010901,0.016175,8.690469e-04,0.000884,0.877593,0.042765,0.000561,0.024192,0.567649,...,0.024495,0.033320,0.000563,0.001764,0.000252,0.018379,0.002685,0.000973,0.003099,0.036323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21082,0.000034,0.003752,0.096772,9.991800e-07,0.044860,0.068827,0.015992,0.000175,0.013986,0.023011,...,0.001641,0.100649,0.002577,0.006282,0.071084,0.033851,0.030482,0.000383,0.000377,0.013612
21083,0.002649,0.017024,0.814692,3.225515e-02,0.020967,0.055764,0.015764,0.049465,0.047083,0.881014,...,0.020583,0.001250,0.000076,0.000148,0.015587,0.004439,0.015767,0.005899,0.007062,0.155187
21084,0.014603,0.010075,0.018993,3.496459e-03,0.002141,0.013637,0.003742,0.001756,0.001122,0.040488,...,0.044033,0.003600,0.001678,0.041600,0.035831,0.003396,0.002919,0.000190,0.003420,0.004164
21085,0.003266,0.012305,0.003702,1.013892e-02,0.023881,0.011510,0.001618,0.021485,0.011895,0.977521,...,0.022565,0.469254,0.014063,0.058853,0.002476,0.000993,0.095780,0.000226,0.044532,0.803760


In [ ]:
y_true = test_df[['PM2.5', 'การเดินทาง', 'กีดขวาง', 'คนจรจัด', 'คลอง', 'ความปลอดภัย',
       'ความสะอาด', 'จราจร', 'ต้นไม้', 'ถนน', 'ทางเท้า', 'ท่อระบายน้ำ',
       'น้ำท่วม', 'ป้าย', 'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม', 'สะพาน',
       'สัตว์จรจัด', 'สายไฟ', 'ห้องน้ำ', 'เสนอแนะ', 'เสียงรบกวน', 'แสงสว่าง']]

In [ ]:
y_true

,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,ความสะอาด,จราจร,ต้นไม้,ถนน,...,ป้ายจราจร,ร้องเรียน,สอบถาม,สะพาน,สัตว์จรจัด,สายไฟ,ห้องน้ำ,เสนอแนะ,เสียงรบกวน,แสงสว่าง
0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21082,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
21083,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
21084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21085,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Create a list of thresholds ranging from 0 to 1 with a step of 0.01
thresholds_ = np.arange(0, 1, 0.01)

best_f1_score = 0
best_threshold = 0

# Iterate over the thresholds and calculate F1-score for each
for threshold in thresholds_:
    # Apply the threshold to y_pred_df
    y_pred_thresholded = output_df.applymap(lambda x: 1 if x >= threshold else 0)

    # Calculate F1-score
    f1 = f1_score(y_true, y_pred_thresholded, average='macro')  # Replace y_true with your true labels

    # Update the best F1-score and threshold if needed
    if f1 > best_f1_score:
        best_f1_score = f1
        best_threshold = threshold

print("Best F1-score:", best_f1_score)
print("Best Threshold:", best_threshold)

Best F1-score: 0.8871536759861308
Best Threshold: 0.36


In [ ]:
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
hl_metric = tfa.metrics.HammingLoss(mode='multilabel', threshold=best_threshold)
hl_metric.update_state(y_true, output_df)
hl_metric.result().numpy()

0.012501778

In [ ]:
y_pred_df = output_df.applymap(lambda x: 1 if x >= best_threshold else 0)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred_df, target_names=cols))

              precision    recall  f1-score   support

       PM2.5       0.71      0.87      0.78       377
  การเดินทาง       0.62      0.43      0.51       168
     กีดขวาง       0.86      0.81      0.84      1625
     คนจรจัด       0.88      0.81      0.84       326
        คลอง       0.94      0.93      0.94      1285
 ความปลอดภัย       0.94      0.93      0.94      2986
   ความสะอาด       0.89      0.93      0.91      1773
       จราจร       0.95      0.91      0.93      1596
      ต้นไม้       0.97      0.97      0.97      1542
         ถนน       0.94      0.97      0.96      7164
     ทางเท้า       0.93      0.96      0.94      2973
 ท่อระบายน้ำ       0.95      0.85      0.90      1861
     น้ำท่วม       0.94      0.95      0.95      1435
        ป้าย       0.96      0.95      0.96      1730
   ป้ายจราจร       0.91      0.92      0.92        92
   ร้องเรียน       0.73      0.64      0.68      2463
      สอบถาม       0.94      0.73      0.82       421
       สะพาน       0.93    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
